# Preprocess IMU

## Modules

In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import dbpd
import numpy as np
import os
import tsdf

from dbpd import DataColumns

## Constants

In [2]:
sensor = 'IMU'

path_to_data = '../../../tests/data/'
input_path = os.path.join(path_to_data, '1.sensor_data')
output_path = os.path.join(path_to_data, '2.preprocessed_data')

meta_filename = f'{sensor}_meta.json'
values_filename = f'{sensor}_samples.bin'
time_filename = f'{sensor}_time.bin'

rotation_units = 'deg/s'
acceleration_units = 'm/s^2'

d_channels_units = {
    DataColumns.ACCELERATION_X: acceleration_units,
    DataColumns.ACCELERATION_Y: acceleration_units,
    DataColumns.ACCELERATION_Z: acceleration_units,
    DataColumns.ROTATION_X: rotation_units,
    DataColumns.ROTATION_Y: rotation_units,
    DataColumns.ROTATION_Z: rotation_units,
}

# filtering
fs = 100
lower_cutoff_frequency = 0.3
filter_order = 4


## Load data

In [7]:
metadata_dict = tsdf.load_metadata_from_path(os.path.join(input_path, meta_filename))
metadata_time = metadata_dict[time_filename]
metadata_samples = metadata_dict[values_filename]
df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df.sample(2)

,time,acceleration_x,acceleration_y,acceleration_z,rotation_x,rotation_y,rotation_z
32853,10.0,297,1210,-1510,15,-21,-1
17843,10.0,310,1542,-1133,16,-21,-3


## Preprocess data

In [8]:
ppp = dbpd.PreprocessingPipelineConfig(
    time_column='time',
    sampling_frequency=fs,
    resampling_frequency=fs,
    gyroscope_units=rotation_units
    )

df['time'] = dbpd.imu_preprocessing.transform_time_array(
    time_array=df['time'],
    scale_factor=1, 
    data_in_delta_time=True)

df = dbpd.imu_preprocessing.resample_data(
    config=ppp,
    time_abs_array=np.array(df['time']),
    values_unscaled=np.array(df[list(d_channels_units.keys())]),
    scale_factors=metadata_samples.scale_factors)

for col in [x for x in d_channels_units.keys() if 'acceleration' in x]:
    for result, side_pass in zip(['filt', 'grav'], ['lp', 'hp']):
        df[f'{result}_{col}'] = dbpd.imu_preprocessing.butterworth_filter(
            config=ppp,
            single_sensor_col=np.array(df[col]),
            order=filter_order,
            cutoff_frequency=lower_cutoff_frequency,
            passband=side_pass
            )
        
    df = df.drop(columns=[col])
    df = df.rename(columns={f'filt_{col}': col})

# Store data

In [10]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

for sensor, units in zip(['acceleration', 'rotation'], [acceleration_units, rotation_units]):
    meta_filename_store = f'{sensor}_meta.json'
    df_sensor = df[['time'] + [x for x in df.columns if sensor in x]]

    metadata_samples.__setattr__('channels', [x for x in df.columns if sensor in x])
    metadata_samples.__setattr__('units', list(np.repeat(units, len(metadata_samples.channels))))
    metadata_samples.__setattr__('meta_filename', meta_filename_store)
    metadata_samples.__setattr__('file_name', meta_filename_store.replace('_meta.json', '_samples.bin'))
    metadata_samples.__setattr__('file_dir_path', output_path)  

    metadata_time.__setattr__('file_dir_path', output_path)
    metadata_time.__setattr__('meta_filename', meta_filename_store)
    metadata_time.__setattr__('file_name', meta_filename_store.replace('_meta.json', '_time.bin'))

    tsdf.write_dataframe_to_binaries(output_path, df_sensor, [metadata_time, metadata_samples])
    tsdf.write_metadata([metadata_time, metadata_samples], meta_filename_store)